In [0]:
# ------------------------------
# Notebook: 03_analysis_kpis (Fully Fixed)
# Purpose: Compute enhanced KPIs: engagement, pass/fail, scores, skills, trends
# ------------------------------

from pyspark.sql.functions import (
    col, lit, sum, avg, countDistinct, when,
    month, weekofyear, corr, regexp_replace, coalesce,
    from_unixtime, to_date
)
from pyspark.sql.types import FloatType

# ------------------------------
# 0️ Load cleaned/master tables
# ------------------------------
sv = spark.table("student_vle")             # student VLE activity
si = spark.table("student_info")           # student info
sa = spark.table("student_assessment")           # student assessment
a  = spark.table("assessments")                  # assessment info
c  = spark.table("courses")                      # course info
vle_info = spark.table("vle")                    # VLE reference table
ocd = spark.table("online_classroom_data") # Online classroom data
df_master = spark.table("student_master")       # master table

# ------------------------------
# 1️ Engagement by activity_type
# Join on code_module + code_presentation only
# ------------------------------
# Replace "code_module" and "code_presentation" with actual column names in sv if different
sv_with_activity = sv.join(
    vle_info.select("code_module", "code_presentation", "activity_type").dropDuplicates(),
    on=[
        sv["code_module"] == vle_info["code_module"],
        sv["code_presentation"] == vle_info["code_presentation"]
    ],
    how="left"
)

# Replace null activity_type with "Unknown"
sv_with_activity = sv_with_activity.withColumn(
    "activity_type",
    coalesce(col("activity_type"), lit("Unknown"))
)

# KPI: total & avg clicks by activity type
df_engagement_by_activity = sv_with_activity.groupBy("activity_type").agg(
    sum("sum_click").alias("total_clicks"),
    avg("sum_click").alias("avg_clicks")
)
display(df_engagement_by_activity.limit(10))

# Engagement per student per activity type
df_engagement_student_activity = sv_with_activity.groupBy("id_student", "activity_type").agg(
    sum("sum_click").alias("total_clicks"),
    avg("sum_click").alias("avg_clicks")
)
display(df_engagement_student_activity.limit(10))

# ------------------------------
# 2️ Pass / Fail rate per module and presentation
# ------------------------------
df_pass_fail_module = df_master.groupBy("code_module").agg(
    countDistinct("id_student").alias("total_students"),
    countDistinct(when(col("final_result") == "Pass", col("id_student"))).alias("passed"),
    countDistinct(when(col("final_result") == "Fail", col("id_student"))).alias("failed"),
    countDistinct(when(col("final_result") == "Withdrawn", col("id_student"))).alias("withdrawn")
).withColumn(
    "pass_rate", col("passed") / col("total_students")
).withColumn(
    "fail_rate", col("failed") / col("total_students")
)
display(df_pass_fail_module.limit(10))

df_pass_fail_presentation = df_master.groupBy("code_module", "code_presentation").agg(
    countDistinct("id_student").alias("total_students"),
    countDistinct(when(col("final_result") == "Pass", col("id_student"))).alias("passed"),
    countDistinct(when(col("final_result") == "Fail", col("id_student"))).alias("failed"),
    countDistinct(when(col("final_result") == "Withdrawn", col("id_student"))).alias("withdrawn")
).withColumn(
    "pass_rate", col("passed") / col("total_students")
).withColumn(
    "fail_rate", col("failed") / col("total_students")
)
display(df_pass_fail_presentation.limit(10))

# ------------------------------
# 3️ Scores by module and presentation
# ------------------------------
df_scores_presentation = df_master.groupBy("code_module", "code_presentation").agg(
    avg("score").alias("avg_score"),
    sum("score").alias("total_score"),
    countDistinct("id_student").alias("students_count")
)
display(df_scores_presentation.limit(10))

# ------------------------------
# 4️ Skills analysis (21st century skills)
# ------------------------------
df_skills = ocd.select(
    avg(regexp_replace(col("sk1_classroom"), ",", ".").cast(FloatType())).alias("avg_SK1"),
    avg(regexp_replace(col("sk2_classroom"), ",", ".").cast(FloatType())).alias("avg_SK2"),
    avg(regexp_replace(col("sk3_classroom"), ",", ".").cast(FloatType())).alias("avg_SK3"),
    avg(regexp_replace(col("sk4_classroom"), ",", ".").cast(FloatType())).alias("avg_SK4"),
    avg(regexp_replace(col("sk5_classroom"), ",", ".").cast(FloatType())).alias("avg_SK5")
)
display(df_skills.limit(10))

# Average scores by highest education
df_skills_education = df_master.groupBy("highest_education").agg(
    avg("score").alias("avg_score_by_education")
)
display(df_skills_education.limit(10))

# Average scores by region
df_skills_region = df_master.groupBy("region").agg(
    avg("score").alias("avg_score_by_region")
)
display(df_skills_region.limit(10))

# ------------------------------
# 5️ Demographic influence on engagement
# ------------------------------
df_demographics_engagement = sv_with_activity.join(si, on="id_student", how="left").groupBy(
    "gender", "age_band", "disability"
).agg(
    sum("sum_click").alias("total_clicks"),
    avg("sum_click").alias("avg_clicks")
)
display(df_demographics_engagement.limit(10))

# ------------------------------
# 6️ Trends over time
# ------------------------------

# Identify actual date column in student_vle
possible_date_cols = ["vle_date", "date", "week_from", "date_id"]
for c in possible_date_cols:
    if c in sv_with_activity.columns:
        date_col = c
        break
else:
    raise ValueError("No valid date column found in sv_with_activity")

# Weekly engagement using the correct column
df_weekly_engagement = sv_with_activity.groupBy(date_col, "activity_type").agg(
    sum("sum_click").alias("total_clicks"),
    avg("sum_click").alias("avg_clicks")
).withColumnRenamed(date_col, "week_date")

display(df_weekly_engagement.limit(10))


# ------------------------------
# 7️ Correlation between engagement and score
# ------------------------------
df_engagement_score = sv_with_activity.groupBy("id_student").agg(
    sum("sum_click").alias("total_clicks")
).join(
    sa.groupBy("id_student").agg(avg("score").alias("avg_score")),
    on="id_student",
    how="left"
)
df_engagement_score.select(corr("total_clicks", "avg_score").alias("clicks_score_correlation")).show()

# ------------------------------
# 8️ Save KPI tables
# ------------------------------
df_engagement_by_activity.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_engagement_kpi")
df_engagement_student_activity.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_engagement_student_activity_kpi")
df_pass_fail_module.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_pass_fail_kpi")
df_pass_fail_presentation.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_pass_fail_presentation_kpi")
df_scores_presentation.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_scores_presentation_kpi")
df_skills.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_skills_kpi")
df_skills_education.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_skills_education_kpi")
df_skills_region.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_skills_region_kpi")
df_demographics_engagement.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_demographics_engagement_kpi")
df_weekly_engagement.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_weekly_engagement_kpi")
df_monthly_scores.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("student_monthly_scores_kpi")

# ------------------------------
# 9️ Display master table for reference
# ------------------------------
display(df_master.limit(10))


activity_type,total_clicks,avg_clicks
glossary,30938723,3.645846109247916
oucontent,39605099,3.7169458709672574
url,38271215,3.727192850639363
homepage,39605099,3.7169458709672574
dualpane,21578149,4.336503995633381
folder,8092058,4.35671741032371
dataplus,18840574,4.316483446996504
resource,39605099,3.7169458709672574
questionnaire,19267666,4.326970396748383
htmlactivity,5281809,4.363836679861099


id_student,activity_type,total_clicks,avg_clicks
683902,oucontent,1203,3.855769230769231
50487,oucollaborate,5021,5.076845298281092
602818,oucollaborate,2837,4.866209262435677
379278,oucontent,3896,5.072916666666667
641547,oucontent,1671,3.9976076555023923
649434,oucontent,3818,4.760598503740648
684594,oucontent,357,2.1377245508982035
682015,oucontent,330,2.5
397506,oucollaborate,9070,3.602065131056394
605756,oucollaborate,2615,4.394957983193278


code_module,total_students,passed,failed,withdrawn,pass_rate,fail_rate
AAA,712,487,91,116,0.6839887640449438,0.12780898876404495
DDD,5848,2227,1390,2078,0.3808139534883721,0.23768809849521202
FFF,7397,2978,1697,2269,0.40259564688387184,0.2294173313505475
BBB,7692,3077,1757,2325,0.4000260010400416,0.22841913676547063
GGG,2525,1118,727,290,0.4427722772277228,0.2879207920792079
EEE,2859,1294,557,697,0.45260580622595314,0.19482336481287163
CCC,4251,1180,781,1884,0.2775817454716537,0.18372147729945895


code_module,code_presentation,total_students,passed,failed,withdrawn,pass_rate,fail_rate
EEE,2013J,1052,482,200,243,0.45817490494296575,0.19011406844106463
CCC,2014B,1936,471,375,898,0.24328512396694216,0.19369834710743802
GGG,2013J,952,451,294,66,0.4737394957983193,0.3088235294117647
DDD,2014B,1228,360,259,490,0.2931596091205212,0.21091205211726385
AAA,2013J,383,258,45,60,0.6736292428198434,0.1174934725848564
AAA,2014J,365,229,46,66,0.6273972602739726,0.12602739726027398
DDD,2013B,1303,456,361,432,0.3499616270145817,0.27705295471987723
EEE,2014B,694,285,164,173,0.4106628242074928,0.23631123919308358
BBB,2014B,1613,561,396,490,0.34779913205207685,0.24550526968381897
FFF,2013J,2283,908,513,675,0.3977222952255804,0.22470433639947437


code_module,code_presentation,avg_score,total_score,students_count
EEE,2013J,76.53044790049007,2.43051825E8,1052
CCC,2014B,77.69095460556606,8.24639994E8,1936
GGG,2013J,81.86290522208039,1.01196795E8,952
DDD,2014B,73.62683070796106,2.62817525E8,1228
AAA,2013J,73.33318259849216,6.9245811E7,383
AAA,2014J,73.37556162304288,6.5357227E7,365
DDD,2013B,73.26708960317383,5.82032441E8,1303
EEE,2014B,77.48959412439858,1.88498707E8,694
BBB,2014B,81.25993447159314,2.11432255E8,1613
FFF,2013J,79.32639298492002,1.112506335E9,2283


...

region,avg_score_by_region
Ireland,77.79758623860778
West Midlands Region,76.66608494641363
North Western Region,76.38254494200204
London Region,77.36418449290771
South East Region,79.01378827142695
South West Region,78.77520977358623
Yorkshire Region,77.20812298227447
Scotland,78.53262720735397
South Region,78.4989651015619
North Region,80.63555714819456


gender,age_band,disability,total_clicks,avg_clicks
M,35-55,N,127615152,3.995097507937435
M,35-55,Y,9849595,3.9908053735871563
F,0-35,Y,13084066,3.351186086961867
M,0-35,Y,22443014,3.8789410945060725
M,55<=,N,5967649,3.792810015946247
M,0-35,N,237818455,3.8840386451126983
F,35-55,N,63376312,3.778568711976238
F,55<=,N,1475237,4.561859702830991
F,0-35,N,97591135,3.5057219765259817
F,35-55,Y,5806228,3.546749775206224


week_date,activity_type,total_clicks,avg_clicks
212,oucollaborate,123587,4.418555595280658
212,url,154403,4.477915373684058
212,forumng,162983,4.415327933248449
212,repeatactivity,47736,6.0
212,questionnaire,89649,5.6532349602724175
212,homepage,162983,4.415327933248449
212,glossary,126600,4.269238551291562
212,subpage,162983,4.415327933248449
212,folder,36307,5.951967213114754
212,resource,162983,4.415327933248449


+------------------------+
|clicks_score_correlation|
+------------------------+
|     0.25544131304984324|
+------------------------+



id_assessment,id_student,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,vle_code_module,vle_code_presentation,id_site,sum_click,vle_date,date_submitted,is_banked,score,assessment_code_module,assessment_code_presentation,assessment_type,assessment_date,weight,course_code_module,course_code_presentation,module_presentation_length,vle_id_site,assessment_date_submitted,assessment_date_assessments,vle_info_id_site,vle_info_code_module,vle_info_code_presentation,activity_type,week_from,week_to
34910,683902,FFF,2014J,M,East Midlands Region,A Level or Equivalent,0-10%,0-35,0,60,N,Pass,FFF,2014J,882549,3,2019-12-28,212,0,78.0,FFF,2014J,CMA,241,0.0,FFF,2014J,269,null,null,null,null,null,null,null,null,null
34910,683902,FFF,2014J,M,East Midlands Region,A Level or Equivalent,0-10%,0-35,0,60,N,Pass,FFF,2014J,883281,2,2019-12-28,212,0,78.0,FFF,2014J,CMA,241,0.0,FFF,2014J,269,null,null,null,null,null,null,null,null,null
34910,683902,FFF,2014J,M,East Midlands Region,A Level or Equivalent,0-10%,0-35,0,60,N,Pass,FFF,2014J,882545,1,2019-12-31,212,0,78.0,FFF,2014J,CMA,241,0.0,FFF,2014J,269,null,null,null,null,null,null,null,null,null
34910,683902,FFF,2014J,M,East Midlands Region,A Level or Equivalent,0-10%,0-35,0,60,N,Pass,FFF,2014J,882674,5,2020-01-05,212,0,78.0,FFF,2014J,CMA,241,0.0,FFF,2014J,269,null,null,null,null,null,null,null,null,null
34910,683902,FFF,2014J,M,East Midlands Region,A Level or Equivalent,0-10%,0-35,0,60,N,Pass,FFF,2014J,883037,2,2020-01-05,212,0,78.0,FFF,2014J,CMA,241,0.0,FFF,2014J,269,null,null,null,null,null,null,null,null,null
34910,683902,FFF,2014J,M,East Midlands Region,A Level or Equivalent,0-10%,0-35,0,60,N,Pass,FFF,2014J,882549,2,2020-01-05,212,0,78.0,FFF,2014J,CMA,241,0.0,FFF,2014J,269,null,null,null,null,null,null,null,null,null
34910,683902,FFF,2014J,M,East Midlands Region,A Level or Equivalent,0-10%,0-35,0,60,N,Pass,FFF,2014J,882549,7,2020-01-05,212,0,78.0,FFF,2014J,CMA,241,0.0,FFF,2014J,269,null,null,null,null,null,null,null,null,null
34910,683902,FFF,2014J,M,East Midlands Region,A Level or Equivalent,0-10%,0-35,0,60,N,Pass,FFF,2014J,882919,1,2020-01-15,212,0,78.0,FFF,2014J,CMA,241,0.0,FFF,2014J,269,null,null,null,null,null,null,null,null,null
34910,683902,FFF,2014J,M,East Midlands Region,A Level or Equivalent,0-10%,0-35,0,60,N,Pass,FFF,2014J,883076,1,2020-01-15,212,0,78.0,FFF,2014J,CMA,241,0.0,FFF,2014J,269,null,null,null,null,null,null,null,null,null
34910,683902,FFF,2014J,M,East Midlands Region,A Level or Equivalent,0-10%,0-35,0,60,N,Pass,FFF,2014J,882549,1,2020-01-15,212,0,78.0,FFF,2014J,CMA,241,0.0,FFF,2014J,269,null,null,null,null,null,null,null,null,null
